# From MLOOps to MLOps

Multi-generative demo which shows a (really bad) comic book generator from voice.

This lifts most content from other notebooks:

- `whisper/whisper-example.ipynb` for the automatic speech recognition
- `natural-language-processing/FLAN-T5-generative-inference.ipynb` for the Text-to-text pipeline
- And the stable diffusion notebooks

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies

IPU Whisper runs faster with the latest features available in SDK > 3.3.0.

In [ ]:
import os
executable_cache_dir = "/storage/mloops-demo/executable-caches"

%env POPTORCH_CACHE_DIR=/storage/mloops-demo/executable-caches
%env POPLAR_EXECUTABLE_CACHE_DIR=/storage/mloops-demo/executable-caches
%env HUGGINGFACE_HUB_CACHE=/storage/mloops-demo/huggingface_caches
%env TRANSFORMERS_CACHE=/storage/mloops-demo/huggingface_caches/checkpoints

In [ ]:
!apt update -y
!apt install -y ffmpeg

In [ ]:
import poptorch
import re
import warnings

sdk_version = !popc --version
if sdk_version and (version := re.search(r'\d+\.\d+\.\d+', sdk_version[0]).group()) >= '3.3':
    print(f"SDK check passed.")
    enable_sdk_features=True
else:
    warnings.warn("SDK versions lower than 3.3 do not support all the functionality in this notebook so performance will be reduced. We recommend you relaunch the Paperspace Notebook with the Pytorch SDK 3.3 image. You can use https://hub.docker.com/r/graphcore/pytorch-early-access", 
                  category=Warning, stacklevel=2)
    enable_sdk_features=False

If the above cell did not pass the SDK check, you can open a runtime with our SDK 3.3.0-EA enabled by clicking the Run on Gradient button below.

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/kC8VBy)


Install the dependencies the notebook needs.

In [ ]:
# Install optimum from source 
%pip install "optimum-graphcore>=0.6, <0.7"
%pip install soundfile==0.12.1 librosa==0.10.0.post2 tokenizers==0.12.1 gradio
%pip install matplotlib fastapi
%matplotlib inline

## Automatic speech recognition with Whisper

We start by importing the required modules, some of which are needed to configure the IPU.


In [ ]:
# Generic imports
from datasets import load_dataset
import matplotlib.pyplot as plt
import librosa
import IPython
import random

# IPU-specific imports
from optimum.graphcore import IPUConfig
from optimum.graphcore.modeling_utils import to_pipelined
from optimum.graphcore.models.whisper import WhisperProcessorTorch

# HF-related imports
from transformers import WhisperForConditionalGeneration

The Whisper model is available on Hugging Face in several sizes, from `whisper-tiny` with 39M parameters to `whisper-large` with 1550M parameters.

We download `whisper-tiny` which we will run using two IPUs.
The [Whisper architecture](https://openai.com/research/whisper) is an encoder-decoder Transformer, with the audio split into 30-second chunks.
For simplicity one IPU is used for the encoder part of the graph and another for the decoder part.
The `IPUConfig` object helps to configure the model to be pipelined across the IPUs.

In [ ]:
model_spec = "openai/whisper-tiny.en"

# Instantiate processor and model
processor = WhisperProcessorTorch.from_pretrained(model_spec)
model = WhisperForConditionalGeneration.from_pretrained(model_spec)

# Adapt whisper-tiny to run on the IPU
ipu_config = IPUConfig(ipus_per_replica=2)
pipelined_model = to_pipelined(model, ipu_config)
pipelined_model = pipelined_model.parallelize(
    for_generation=True, 
    use_cache=True, 
    batch_size=1, 
    max_length=250,
    on_device_generation_steps=16, 
    use_encoder_output_buffer=enable_sdk_features
).half()

def whisper_transcribe(data, rate):
    input_features = processor(data, return_tensors="pt", sampling_rate=rate).input_features.half()

    # This triggers a compilation, unless a precompiled model is available.
    sample_output = pipelined_model.generate(
        input_features,
        use_cache=True,
        do_sample=False,
        max_length=448, 
        min_length=3)
    transcription = processor.batch_decode(sample_output, skip_special_tokens=True)[0]
    return transcription

Now we can load the dataset and process an example audio file.
If precompiled models are not available, then the first run of the model triggers two graph compilations.
This means that our first test transcription could take a minute or two to run, but subsequent runs will be much faster.

In [ ]:
# load the dataset and read an example sound file
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
test_sample = ds[2]
sample_rate = test_sample['audio']['sampling_rate']


test_transcription = whisper_transcribe(test_sample["audio"]["array"], sample_rate)
test_transcription

In the next cell, we compare the expected text from the dataset with the transcribed result from the model.
There will typically be some small differences, but even `whisper-tiny` does a great job! It even adds punctuation.

You can listen to the audio and compare the model result yourself using the controls below.

In [ ]:
print(f"Expected: {test_sample['text']}\n")
print(f"Transcribed: {test_transcription}")


The model only needs to be compiled once. Subsequent inferences will be much faster.
In the cell below, we repeat the exercise but with a random example from the dataset.

You might like to re-run this next cell multiple times to get different comparisons.

In [ ]:
idx = random.randint(0, ds.num_rows - 1)
data = ds[idx]["audio"]["array"]

print(f"Example #{idx}\n")
print(f"Expected: {ds[idx]['text']}\n")
print(f"Transcribed: {whisper_transcribe(data, sample_rate)}")

IPython.display.Audio(data, rate=sample_rate, autoplay=True)

# Running Flan-T5

In this demo we use a Generative text model to refine our prompts for the Stable diffusion.

In December 2022 Google published [Scaling Instruction-Finetuned Language Models](https://arxiv.org/abs/2210.11416) in which they perform extensive fine-tuning for a broad collection of tasks across a variety of models (PaLM, T5, U-PaLM). Part of this publication was the release of Flan-T5 checkpoints, "which achieve strong few-shot performance" with relatively modest parameter counts "even compared to much larger models" like the largest members of the GPT family.

The cell below uses the Flan-T5 checkpoints from the Hugging Face Hub and uses the `transformers` library.

In [ ]:
num_t5_ipus=4

from optimum.graphcore import pipeline, IPUConfig

size = {4: "large", 16: "xl"}
flan_t5 = pipeline(
    "text2text-generation",
    model=f"google/flan-t5-{size[num_t5_ipus]}",
    ipu_config=IPUConfig.from_pretrained(
        f"Graphcore/t5-{size[num_t5_ipus]}-ipu", executable_cache_dir=executable_cache_dir
    ),
    max_input_length=896,
)

questions = [
    "Solve the following equation for x: x^2 - 9 = 0",
    "At what temperature does nitrogen freeze?",
    "In order to reduce symptoms of asthma such as tightness in the chest, wheezing, and difficulty breathing, what do you recommend?",
    "Which country is home to the tallest mountain in the world?"
]
for out in flan_t5(questions):
    print(out)

The examples below are taken from the `natural-language-processing/FLAN-T5-Generative-inference` notebook:

In [ ]:
sentiment_analysis = (
    "Review: It gets too hot, the battery only can last 4 hours. Sentiment: Negative\n"
    "Review: Nice looking phone. Sentiment: Positive\n"
    "Review: Sometimes it freezes and you have to close all the open pages and then reopen where you were. Sentiment: Negative\n"
    "Review: Wasn't that impressed, went back to my old phone. Sentiment:"
)
advanced_ner = """Microsoft Corporation is a company that makes computer software and video games. Bill Gates and Paul Allen founded the company in 1975
[Company]: Microsoft, [Founded]: 1975, [Founders]: Bill Gates, Paul Allen

Amazon.com, Inc., known as Amazon , is an American online business and cloud computing company. It was founded on July 5, 1994 by Jeff Bezos
[Company]: Amazon, [Founded]: 1994, [Founders]: Jeff Bezos

Apple Inc. is a multinational company that makes personal computers, mobile devices, and software. Apple was started in 1976 by Steve Jobs and Steve Wozniak."""

context = 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24-10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'
question = "Which NFL team represented the AFC at Super Bowl 50?"
contextual_q_a = f"{context} {question}"

intent_classification = """[Text]: I really need to get a gym membership, I'm exhausted.
[Intent]: get gym membership

[Text]: What do I need to make a carbonara?
[Intent]: cook carbonara

[Text]: I need all these documents sorted and filed by Monday.
[Intent]:"""

summarization="""
Document: Firstsource Solutions said new staff will be based at its Cardiff Bay site which already employs about 800 people.
The 300 new jobs include sales and customer service roles working in both inbound and outbound departments.
The company's sales vice president Kathryn Chivers said: "Firstsource Solutions is delighted to be able to continue to bring new employment to Cardiff."
Summary: Hundreds of new jobs have been announced for a Cardiff call centre.

Document: The visitors raced into a three-goal first-half lead at Hampden.
Weatherson opened the scoring with an unstoppable 15th-minute free-kick, and he made it 2-0 in the 27th minute.
Matt Flynn made it 3-0 six minutes later with a fine finish.
Queen's pulled a consolation goal back in stoppage time through John Carter.
Summary: Peter Weatherson netted a brace as Annan recorded only their second win in eight matches.

Document: Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday.
Detectives said three firearms, ammunition and a five-figure sum of money were recovered.
A 26-year-old man who was arrested and charged appeared at Edinburgh Sheriff Court on Thursday.
Summary:
"""
text_classification_1 = """A return ticket is better value than a single.
topic: travel cost

You can start from the basic stitches, and go from there.
topic: learning knitting

The desk which I bought yesterday is very big.
topic: furniture size

George Washington was president of the United States from 1789 to 1797.
topic:"""
text_classification_2 = """FLAN-T5 was released in the paper Scaling Instruction-Finetuned Language Models - it is an enhanced version of T5 that has been finetuned in a mixture of tasks.
keywords: released, enhanced, finetuned

The IPU, or Intelligence Processing Unit, is a highly flexible, easy-to-use parallel processor designed from the ground up for AI workloads.
keywords: processor, AI

Paperspace is the platform for AI developers. providing the speed and scale needed to take AI models from concept to production.
keywords:"""

flan_t5_examples = questions + [sentiment_analysis, advanced_ner, contextual_q_a, summarization, intent_classification, text_classification_1, text_classification_2]

# Running stable diffusion

We also instantiate stable diffusion for doing text to image generation.

Stable diffusion is chosen to run on 8 IPUs:

In [ ]:
number_of_stable_diffusion_ipus = 8

As was the case for the other models, stable diffusion is loaded onto IPUs in a few lines of code:

In [ ]:
import torch
from diffusers import DPMSolverMultistepScheduler

from optimum.graphcore.diffusers import get_default_ipu_configs, INFERENCE_ENGINES_TO_MODEL_NAMES, IPUStableDiffusionPipeline


engine = "stable-diffusion-v1-5"  # maps to "runwayml/stable-diffusion-v1-5"
model_name = INFERENCE_ENGINES_TO_MODEL_NAMES[engine]
image_width = os.getenv("STABLE_DIFFUSION_TXT2IMG_DEFAULT_WIDTH", default=512)
image_height = os.getenv("STABLE_DIFFUSION_TXT2IMG_DEFAULT_HEIGHT", default=512)

unet_ipu_config, text_encoder_ipu_config, vae_ipu_config, safety_checker_ipu_config = \
get_default_ipu_configs(
    engine=engine, width=image_width, height=image_height, n_ipu=number_of_stable_diffusion_ipus, 
    executable_cache_dir=executable_cache_dir 
)
pipe = IPUStableDiffusionPipeline.from_pretrained(
    model_name,
    revision="fp16", 
    torch_dtype=torch.float16,
    requires_safety_checker=False,
    unet_ipu_config=unet_ipu_config,
    text_encoder_ipu_config=text_encoder_ipu_config,
    vae_ipu_config=vae_ipu_config,
    safety_checker_ipu_config=safety_checker_ipu_config
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

The model is then compiled by running one inference loop:

In [ ]:
pipe("apple", height=image_height, width=image_width, guidance_scale=7.5);

# Assembling models in a single application

In this section we will assembling the models into a simple Fast API web site which will have several of our apps.

Fist we can check that our models are ready to run, we'll see below which models are running:

In [ ]:
!gc-monitor --no-card-info

Test the audio model:

In [ ]:
idx = 58

transcription = whisper_transcribe(data, sample_rate)
print(f"Example #{idx}\n")
print(f"Expected: {ds[idx]['text']}\n")
print(f"Transcribed: {transcription}")

IPython.display.Audio(data, rate=sample_rate, autoplay=True)

In [ ]:
# A little helper function for turning Librispeech IDs into audio samples

def sample_audio(audio_prompts):
    f, m, *l = audio_prompts
    audio_prompts = f, min(m, len(ds)), *l
    return np.concatenate([ds[idx]["audio"]["array"] for idx in range(*audio_prompts)])

Test linking audio and image models:

In [ ]:
idx = 58
data = ds[idx]["audio"]["array"]
IPython.display.Audio(data, rate=sample_rate, autoplay=True)
transcription = whisper_transcribe(data, sample_rate)
print(transcription)
out = pipe(transcription, height=image_height, width=image_width, guidance_scale=7.5)
out.images[0]

Now the output from the ASR model does not make a good input to stable diffusion prompts.

To do that, we use FLAN-T5 to reformat our sentences, into stable diffusion prompts:

In [ ]:


LLM_prompt = """
[Long text]: {previous_story}
[source] {previous_sentence} [description]:
"""

LLM_continue = """
{previous_story}
Continue the story without repetition:
"""

def LLM_to_sd_prompts(generated_text, prompt_for_sd=LLM_prompt, prompt_to_continue=LLM_continue, story_steps=3, llm_replace=False):
    print("Continuing story")
    for i in range(story_steps):
        arg = dict(
            previous_story = " ".join(generated_text),
            previous_sentence = generated_text[-1],
        )
        prompt = prompt_to_continue.format(**arg)
        out_text = flan_t5(prompt)
        generated_text.append(out_text[0]['generated_text'])
    
    print("Transformming Stable diffusion prompts")
    transformed_text = []
    if llm_replace:
        for i, sentence in enumerate(generated_text):
            arg = dict(
                previous_story = " ".join(generated_text[:i]),
                previous_sentence = generated_text[i],
            )
            prompt = prompt_for_sd.format(**arg)
            out_text = flan_t5(prompt)
            transformed_text.append(out_text[0]['generated_text'])
    else:
        transformed_text.extend(generated_text)

    return transformed_text

LLM_to_sd_prompts([transcription], LLM_prompt, LLM_continue, llm_replace=True)

In [ ]:
import numpy as np
audio_prompts = (10, 30)
# style_prompt = "japanese, manga, high resolution, dynamic."
style_prompt = "modern art, smooth vibes"


def generate_comic(audio, style_prompt, prompt_for_sd=LLM_prompt, prompt_to_continue=LLM_continue, max_images=None, sample_rate=sample_rate, generate_image=True, llm_replace=False):
    generated_text = []
    print("Transcribing audio")
    
    transcription = ""
    max_model_samples = sample_rate * 30
    for i in range(0, len(audio), max_model_samples):
        print(f"  Audio was {len(audio) // sample_rate} seconds long, processing part: {i/max_model_samples+1} in chunks of 30s")
        transcription += whisper_transcribe(audio[i:(i+1)*max_model_samples], sample_rate)

    generated_text = [f"{t}. " for t in transcription.split(".")]
    generated_text = LLM_to_sd_prompts(generated_text, prompt_for_sd=prompt_for_sd, prompt_to_continue=prompt_to_continue, story_steps=3, llm_replace=llm_replace)
    images = []
    if not generate_image:
        return transcription, generated_text, images, audio
    print("generating images")
    nstep = 0
    for prompt in generated_text:
        out = pipe(prompt + style_prompt, height=image_height, width=image_width, guidance_scale=7.5)
        images.append(out.images[0])
        nstep +=1
        if not (max_images is None) and nstep >= max_images:
            break
    print(generated_text)
    return transcription, generated_text, images, audio

transcription, generated_text, images, data = generate_comic(sample_audio(audio_prompts), style_prompt, max_images=4)
print("rendering audio")
print(transcription)
IPython.display.Audio(data, rate=sample_rate, autoplay=False)

In [ ]:
from matplotlib import pyplot as plt
import pathlib

def comic_book_plotter(generated_text, images, style_prompt, image_per_page=6, line_break=40, fig_size=(8,15)):
    name = style_prompt.replace(" ","_").replace(".","_").replace(".","_").strip("_")
    comic_hash = hash("\n".join(generated_text)) + sum(hash(image.tobytes()) for image in images)
    figs = []
    paths = []
    for page_num, id_start in enumerate(range(0,len(generated_text), image_per_page)):
        comic_text = generated_text[id_start:id_start+image_per_page]
        comic_images = images[id_start:id_start+image_per_page]
        if not comic_images:
            break
        fig, axs = plt.subplots(image_per_page//2, 2)
        figs.append(fig)
        fig.set_size_inches(*fig_size)
        
        for image, prompt, ax in zip(comic_images, comic_text,axs.flatten()):
            ax.imshow(image)
            breaks = [0] + [prompt.find(" ", i) for i in range(line_break, len(prompt), line_break)] + [-1]
            formatted_prompt = "\n".join(prompt[i:j] for i, j in zip(breaks[:-1], breaks[1:]))
            ax.set_title(f"{formatted_prompt}")
            ax.axis("off")
        fig.suptitle(f"Style prompt: '{style_prompt}' page {page_num+1}", y=1.0)
        fig.tight_layout()

        pathlib.Path("/storage/comics/").mkdir(exist_ok=True)
        image_path = f"/storage/comics/whisper_to_image_{name}_{comic_hash}-page-{page_num+1}.png"
        paths.append(image_path)
        fig.savefig(image_path, dpi=150)
 
    return figs, paths

fig, image_path = comic_book_plotter(generated_text, images, style_prompt)

In [ ]:
import gradio as gr
import librosa

def mic_to_text(rate_and_audio, uploaded_sound):
    if rate_and_audio is None:
        rate_and_audio = uploaded_sound
    source_sample_rate, source_audio= rate_and_audio
    model_sr=16000
    audio = librosa.resample(y=source_audio.astype(float)/np.iinfo(source_audio.dtype).max, orig_sr=source_sample_rate, target_sr=model_sr)
    transcription = ""
    max_model_samples = model_sr * 30
    for i in range(0, len(audio), max_model_samples):
        print(f"  Audio was {len(audio) // sample_rate} seconds long, truncating at 30s")
        transcription += whisper_transcribe(audio[i:(i+1)*max_model_samples], sample_rate)
    return transcription


def text_to_text(prompt):
    return flan_t5(prompt)[0]["generated_text"]



def text_to_image(prompt, n_repeats=4):
    return [pipe(prompt, height=image_height, width=image_width, guidance_scale=7.5).images[0] for _ in range(n_repeats)]



def transcribe_to_comic(rate_and_audio, style_prompt, max_images, transform_sentence_to_prompts, llm_to_sd_prompt, uploaded_sound):
    if rate_and_audio is None:
        rate_and_audio = uploaded_sound
    sample_rate, audio= rate_and_audio
    target_sr=16000
    resample_audio = librosa.resample(y=audio.astype(float)/np.iinfo(audio.dtype).max, orig_sr=sample_rate, target_sr=target_sr)
    transcription, generated_text, images, data = generate_comic(
        resample_audio, style_prompt, sample_rate=target_sr, max_images=max_images, prompt_for_sd=llm_to_sd_prompt, llm_replace=transform_sentence_to_prompts
    )
    figs, paths = comic_book_plotter(generated_text, images, style_prompt, image_per_page=2, line_break=40, fig_size=(8,8))
    return transcription, paths, "\n".join(generated_text)

import soundfile

def to_sample_audio_file(libri_id):
    audio = sample_audio(libri_id)
    file = f"librispeech_validation_{libri_id[0]}-{libri_id[1]}.wav"
    soundfile.write(file, audio, 16000)
    return file


In [ ]:
audio_length = 15
audio_examples= [
    to_sample_audio_file((i, i+audio_length)) for i in range(0, len(ds), audio_length)
]

In [ ]:

comic_book_app = gr.Interface(
    fn=transcribe_to_comic,
    inputs=[
        gr.Audio(source="microphone", type="numpy"),
        "text",
        gr.Slider(value=10),
        gr.Checkbox(value=False),
        gr.TextArea(value=LLM_prompt),
        gr.Audio(source="upload", type="numpy"),
    ], 
    outputs=["text", gr.Gallery(min_width=800, preview=False), gr.TextArea()],
    examples=[
        list(e) for e in zip(audio_examples,
        [
            "Aquarel, calm.",
            "Dali, surrealistic, dreamy.",
            "Manga, high resolution, dynamic.",
            "Turner, oil painting.",
        ]* len(audio_examples),
        [None] * len(audio_examples),
        [None] * len(audio_examples),
        [None] * len(audio_examples),
     )
    ]
).queue()

asr_app = gr.Interface(
    fn=mic_to_text,
    inputs=[
        gr.Audio(source="microphone", type="numpy"),
        gr.Audio(source="upload", type="numpy"),
    ], 
    outputs="text",
    examples=[[e, None] for e in audio_examples],
).queue()

text_app = gr.Interface(
    fn=text_to_text,
    inputs="text", 
    outputs="text",
    examples=flan_t5_examples,
).queue()

image_app = gr.Interface(
    fn=text_to_image,
    inputs="text",
    outputs=gr.Gallery(preview=False),
    examples=[
        "A digital illustration of a steampunk library with clockwork machines, 4k, detailed, trending in artstation, fantasy vivid colors",
        "A digital illustration of a steampunk flying machine in the sky with cogs and mechanisms, 4k, detailed, trending in artstation, fantasy vivid colors",
        "A digital Illustration of the Babel tower, 4k, detailed, trending in artstation, fantasy vivid colors",
        "A medieval town with disco lights and a fountain, by Josef Thoma, matte painting trending on artstation HQ, concept art",
        "Editorial Style Photo, (Low Angle|Eye Level), ${THING YOU WANT}, Task Lighting, {MATERIALS}, {STYLE ADJECTIVES}, Symmetric, 4k",
    ],
    
).queue()

In [ ]:
from fastapi import FastAPI
import uvicorn
from fastapi.responses import HTMLResponse
app = FastAPI()


@app.get("/", response_class=HTMLResponse)
def read_main():
    return """<!DOCTYPE html>
<html>
<head>
<title>Welcome to the MLOops demo</title>
</head>
<body>

<h1>Welcome to the MLOops demo!</h1>

<p>
This is a collection of Gradio apps demonstrating:

<ul>
<li><a href="/voice">Voice transcription with whisper</a></li>
<li><a href="/text">Text to text generation with T5-large</a></li>
<li><a href="/image">Text to image Stable diffusion</a></li>
</ul>

And all those models are assembled into...
<ul>
<li><a href="/comic">A very bad comic book generator</a></li>
</ul>

</p>
</body>
</html>"""


app = gr.mount_gradio_app(app, comic_book_app, path="/comic")
app = gr.mount_gradio_app(app, text_app, path="/text")
app = gr.mount_gradio_app(app, image_app, path="/image")
app = gr.mount_gradio_app(app, asr_app, path="/voice")


In [ ]:
import nest_asyncio
nest_asyncio.apply()
# Needed for coroutine's in Notebooks
import asyncio
loop = asyncio.get_event_loop()
print("got event loop")

running_app = loop.run_until_complete(
    uvicorn.run(app, host="0.0.0.0", port=int(os.getenv('APP_PORT', 7860)))
)


# Try a model for prompting

In [ ]:
magic_prompt = pipeline('text-generation', model="Gustavosta/MagicPrompt-Stable-Diffusion", ipu_config="Graphcore/gpt2-small-ipu")
magic_prompt

In [ ]:
magic_prompt("this is a file")

In [ ]:
flan_t5.model.detachFromDevice()

In [ ]:
magic_prompt.model.detachFromDevice()